In [164]:
import pandas as pd
import numpy as np
import json
import os
import joblib
import re
import nltk
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from torch.utils.data import DataLoader, TensorDataset
from datasets import Dataset
from tqdm.auto import tqdm
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix
from textblob import TextBlob

In [165]:
folder_path = 'E:\\Case Comp\\NEST\\Inference\\'

In [166]:

parquet_file_path = folder_path + 'Usecase_3_.csv'
# usecase_3.to_parquet(parquet_file_path, index=False)

# Load the Parquet file back into a DataFrame
data = pd.read_csv(parquet_file_path)
data.head()

,Unnamed: 0.1,Unnamed: 0,NCT Number,Study Title,Study URL,Acronym,Brief Summary,Study Results,Conditions,Interventions,...,Study Design,Other IDs,Start Date,Primary Completion Date,Completion Date,First Posted,Results First Posted,Last Update Posted,Locations,Study Documents
0,0,0,NCT03162666,Patient Outcomes Using an Expandable Spacer,https://clinicaltrials.gov/study/NCT03162666,NaN,This study is a post-market clinical follow-up...,NO,Degenerative Disc Disease,DEVICE: ALTERA,...,Observational Model: |Time Perspective: p,RGC16-001,2/28/17,11-06-20,11-06-20,5/22/17,NaN,8/23/21,"Rush University Medical Center, Chicago, Illin...",NaN
1,1,1,NCT04312048,the Effect of Isosorbide Mononitrate in Reduci...,https://clinicaltrials.gov/study/NCT04312048,NaN,The aim of the study is to evaluate the effica...,NO,IUD Insertion Pain,DRUG: Isosorbide mononitrate|DRUG: placebo,...,Allocation: RANDOMIZED|Intervention Model: PAR...,249,4/15/20,10/25/20,11/30/20,3/18/20,NaN,1/20/21,"Ahmed Samy, Giza, 11231, Egypt",NaN
2,2,2,NCT03144778,Durvalumab With or Without Tremelimumab in Tre...,https://clinicaltrials.gov/study/NCT03144778,NaN,This phase I trial studies how well durvalumab...,NO,Stage II Oropharyngeal Squamous Cell Carcinoma...,BIOLOGICAL: Durvalumab|BIOLOGICAL: Tremelimumab,...,Allocation: RANDOMIZED|Intervention Model: PAR...,2016-0805|NCI-2018-01199|2016-0805,07-12-17,3/15/21,3/15/21,05-09-17,NaN,3/18/21,"M D Anderson Cancer Center, Houston, Texas, 77...",NaN
3,3,3,NCT01592721,Radiation and Cetuximab Plus Intratumoral EGFR...,https://clinicaltrials.gov/study/NCT01592721,NaN,The incorporation of novel targeted therapies ...,YES,Squamous Cell Carcinoma|Head and Neck Cancer,BIOLOGICAL: EGFR Antisense DNA,...,Allocation: NA|Intervention Model: SINGLE_GROU...,CTRC 11-47|HSC20120131H,2013-04,02-01-22,02-01-22,05-07-12,9/21/22,9/21/22,"University of Pittsburgh, Pittsburgh, Pennsylv...","Study Protocol and Statistical Analysis Plan, ..."
4,4,4,NCT04253613,Laser Biostimulation in Periodontal Treatment,https://clinicaltrials.gov/study/NCT04253613,NaN,"The use of nonsurgical periodontal treatment, ...",NO,Periodontal Inflammation|Periodontal Diseases,NaN,...,Observational Model: |Time Perspective: p,OMU KAEK2012/49,1/15/13,6/15/14,6/15/14,02-05-20,NaN,07-06-22,NaN,NaN


In [167]:
data.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'NCT Number', 'Study Title', 'Study URL',
       'Acronym', 'Brief Summary', 'Study Results', 'Conditions',
       'Interventions', 'Primary Outcome Measures',
       'Secondary Outcome Measures', 'Other Outcome Measures', 'Sponsor',
       'Collaborators', 'Sex', 'Age', 'Phases', 'Enrollment', 'Funder Type',
       'Study Type', 'Study Design', 'Other IDs', 'Start Date',
       'Primary Completion Date', 'Completion Date', 'First Posted',
       'Results First Posted', 'Last Update Posted', 'Locations',
       'Study Documents'],
      dtype='object')

In [168]:
cols_to_drop = [col for col in data.columns if col.startswith('Unnamed')]
data.drop(columns=cols_to_drop, inplace=True)

In [169]:
# data = data.head(100)

In [170]:
data.head()

,NCT Number,Study Title,Study URL,Acronym,Brief Summary,Study Results,Conditions,Interventions,Primary Outcome Measures,Secondary Outcome Measures,...,Study Design,Other IDs,Start Date,Primary Completion Date,Completion Date,First Posted,Results First Posted,Last Update Posted,Locations,Study Documents
0,NCT03162666,Patient Outcomes Using an Expandable Spacer,https://clinicaltrials.gov/study/NCT03162666,NaN,This study is a post-market clinical follow-up...,NO,Degenerative Disc Disease,DEVICE: ALTERA,"Change in Radiographic Analysis, Global and Se...","Complications, Surgical or device related comp...",...,Observational Model: |Time Perspective: p,RGC16-001,2/28/17,11-06-20,11-06-20,5/22/17,NaN,8/23/21,"Rush University Medical Center, Chicago, Illin...",NaN
1,NCT04312048,the Effect of Isosorbide Mononitrate in Reduci...,https://clinicaltrials.gov/study/NCT04312048,NaN,The aim of the study is to evaluate the effica...,NO,IUD Insertion Pain,DRUG: Isosorbide mononitrate|DRUG: placebo,"pain during IUD insertion, intensity of patien...","duration of IUD insertion, duration of IUD ins...",...,Allocation: RANDOMIZED|Intervention Model: PAR...,249,4/15/20,10/25/20,11/30/20,3/18/20,NaN,1/20/21,"Ahmed Samy, Giza, 11231, Egypt",NaN
2,NCT03144778,Durvalumab With or Without Tremelimumab in Tre...,https://clinicaltrials.gov/study/NCT03144778,NaN,This phase I trial studies how well durvalumab...,NO,Stage II Oropharyngeal Squamous Cell Carcinoma...,BIOLOGICAL: Durvalumab|BIOLOGICAL: Tremelimumab,"Change of CD8+ tumor infiltrating lymphocytes,...",Incidence of adverse events and serious advers...,...,Allocation: RANDOMIZED|Intervention Model: PAR...,2016-0805|NCI-2018-01199|2016-0805,07-12-17,3/15/21,3/15/21,05-09-17,NaN,3/18/21,"M D Anderson Cancer Center, Houston, Texas, 77...",NaN
3,NCT01592721,Radiation and Cetuximab Plus Intratumoral EGFR...,https://clinicaltrials.gov/study/NCT01592721,NaN,The incorporation of novel targeted therapies ...,YES,Squamous Cell Carcinoma|Head and Neck Cancer,BIOLOGICAL: EGFR Antisense DNA,"Toxicity Rate, This is a 2-stage clinical tria...","Tumor Response, Clinical secondary endpoints i...",...,Allocation: NA|Intervention Model: SINGLE_GROU...,CTRC 11-47|HSC20120131H,2013-04,02-01-22,02-01-22,05-07-12,9/21/22,9/21/22,"University of Pittsburgh, Pittsburgh, Pennsylv...","Study Protocol and Statistical Analysis Plan, ..."
4,NCT04253613,Laser Biostimulation in Periodontal Treatment,https://clinicaltrials.gov/study/NCT04253613,NaN,"The use of nonsurgical periodontal treatment, ...",NO,Periodontal Inflammation|Periodontal Diseases,NaN,"IL-1β level in GCF, IL-1β is a cytokine presen...",NaN,...,Observational Model: |Time Perspective: p,OMU KAEK2012/49,1/15/13,6/15/14,6/15/14,02-05-20,NaN,07-06-22,NaN,NaN


In [171]:
# ✅ Use GPU if available
device = 0 if torch.cuda.is_available() else -1
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)

# ✅ Define categories
categories = ["Oncology", "Non-Oncology"]

# ✅ Convert Pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(data)

# ✅ Batch processing function (Uses GPU)
def batch_classify(batch, threshold=0.7):
    """Classifies a batch of conditions into predefined categories."""
    results = classifier(batch['Conditions'], candidate_labels=categories, batch_size=64)

    classified_labels = []
    for classification in results:
        best_label = classification['labels'][0]
        best_score = classification['scores'][0]
        classified_labels.append(best_label if best_score >= threshold else "Other Rare or Unclassified")

    return {"Conditions_Category": classified_labels}

# ✅ Optimized batch processing with map function
batch_size = 512
dataset = dataset.map(batch_classify, batched=True, batch_size=batch_size, remove_columns=['Conditions'])

# Convert back to Pandas DataFrame
data = pd.concat([data, dataset.to_pandas()], axis=1)

# ✅ Show Completion Status
total_rows = len(data)
classified_rows = data['Conditions_Category'].notnull().sum()
completion_percentage = (classified_rows / total_rows) * 100
print(f"✅ Classification Completed: {completion_percentage:.2f}% of rows processed.") # Use print instead of tqdm.write for cleaner output

# ✅ Display Sample Results
print(data[['Conditions', 'Conditions_Category']].head())

Device set to use cpu


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

✅ Classification Completed: 100.00% of rows processed.
                                          Conditions Conditions_Category
0                          Degenerative Disc Disease        Non-Oncology
1                                 IUD Insertion Pain        Non-Oncology
2  Stage II Oropharyngeal Squamous Cell Carcinoma...            Oncology
3       Squamous Cell Carcinoma|Head and Neck Cancer            Oncology
4      Periodontal Inflammation|Periodontal Diseases        Non-Oncology


In [172]:
def drop_duplicate_columns(df):
    df_T = df.T.drop_duplicates().T  # Transpose, drop duplicates, and transpose back
    return df_T

data = drop_duplicate_columns(data)

In [173]:
data.head()

,NCT Number,Study Title,Study URL,Acronym,Brief Summary,Study Results,Conditions,Interventions,Primary Outcome Measures,Secondary Outcome Measures,...,Other IDs,Start Date,Primary Completion Date,Completion Date,First Posted,Results First Posted,Last Update Posted,Locations,Study Documents,Conditions_Category
0,NCT03162666,Patient Outcomes Using an Expandable Spacer,https://clinicaltrials.gov/study/NCT03162666,NaN,This study is a post-market clinical follow-up...,NO,Degenerative Disc Disease,DEVICE: ALTERA,"Change in Radiographic Analysis, Global and Se...","Complications, Surgical or device related comp...",...,RGC16-001,2/28/17,11-06-20,11-06-20,5/22/17,NaN,8/23/21,"Rush University Medical Center, Chicago, Illin...",NaN,Non-Oncology
1,NCT04312048,the Effect of Isosorbide Mononitrate in Reduci...,https://clinicaltrials.gov/study/NCT04312048,NaN,The aim of the study is to evaluate the effica...,NO,IUD Insertion Pain,DRUG: Isosorbide mononitrate|DRUG: placebo,"pain during IUD insertion, intensity of patien...","duration of IUD insertion, duration of IUD ins...",...,249,4/15/20,10/25/20,11/30/20,3/18/20,NaN,1/20/21,"Ahmed Samy, Giza, 11231, Egypt",NaN,Non-Oncology
2,NCT03144778,Durvalumab With or Without Tremelimumab in Tre...,https://clinicaltrials.gov/study/NCT03144778,NaN,This phase I trial studies how well durvalumab...,NO,Stage II Oropharyngeal Squamous Cell Carcinoma...,BIOLOGICAL: Durvalumab|BIOLOGICAL: Tremelimumab,"Change of CD8+ tumor infiltrating lymphocytes,...",Incidence of adverse events and serious advers...,...,2016-0805|NCI-2018-01199|2016-0805,07-12-17,3/15/21,3/15/21,05-09-17,NaN,3/18/21,"M D Anderson Cancer Center, Houston, Texas, 77...",NaN,Oncology
3,NCT01592721,Radiation and Cetuximab Plus Intratumoral EGFR...,https://clinicaltrials.gov/study/NCT01592721,NaN,The incorporation of novel targeted therapies ...,YES,Squamous Cell Carcinoma|Head and Neck Cancer,BIOLOGICAL: EGFR Antisense DNA,"Toxicity Rate, This is a 2-stage clinical tria...","Tumor Response, Clinical secondary endpoints i...",...,CTRC 11-47|HSC20120131H,2013-04,02-01-22,02-01-22,05-07-12,9/21/22,9/21/22,"University of Pittsburgh, Pittsburgh, Pennsylv...","Study Protocol and Statistical Analysis Plan, ...",Oncology
4,NCT04253613,Laser Biostimulation in Periodontal Treatment,https://clinicaltrials.gov/study/NCT04253613,NaN,"The use of nonsurgical periodontal treatment, ...",NO,Periodontal Inflammation|Periodontal Diseases,NaN,"IL-1β level in GCF, IL-1β is a cytokine presen...",NaN,...,OMU KAEK2012/49,1/15/13,6/15/14,6/15/14,02-05-20,NaN,07-06-22,NaN,NaN,Non-Oncology


In [174]:
# Combine all text attributes into a single column 'Unstructured'
data["Outcomes"] = data[
    ["Primary Outcome Measures", "Secondary Outcome Measures", "Other Outcome Measures"]
].astype(str).agg(" [SEP] ".join, axis=1)

In [175]:
data.columns

Index(['NCT Number', 'Study Title', 'Study URL', 'Acronym', 'Brief Summary',
       'Study Results', 'Conditions', 'Interventions',
       'Primary Outcome Measures', 'Secondary Outcome Measures',
       'Other Outcome Measures', 'Sponsor', 'Collaborators', 'Sex', 'Age',
       'Phases', 'Enrollment', 'Funder Type', 'Study Type', 'Study Design',
       'Other IDs', 'Start Date', 'Primary Completion Date', 'Completion Date',
       'First Posted', 'Results First Posted', 'Last Update Posted',
       'Locations', 'Study Documents', 'Conditions_Category', 'Outcomes'],
      dtype='object')

In [176]:
# Load Fine-Tuned Model and Tokenizer

model_path = folder_path+ "ClinicalBioBertFinal/study_title_fine_tuned_clinicalbiobert_v3"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Set Device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # Set model to evaluation mode

# Function to tokenize input texts
def encode_texts(texts, tokenizer, max_length=56):
    encodings = tokenizer(
        texts,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    return encodings

# Function for Model Inference
def predict(data, tokenizer, model, device, text_column="Study Title", max_length=56, batch_size=16):
    # Ensure column exists
    if text_column not in data.columns:
        raise KeyError(f"Column '{text_column}' not found in DataFrame. Available columns: {list(data.columns)}")

    # Convert DataFrame column to list
    texts = list(data[text_column].astype(str))
    nct_numbers = list(data["NCT Number"].astype(str))  # Ensure ID column is string

    # Tokenize texts
    encodings = encode_texts(texts, tokenizer, max_length)
    input_ids = encodings["input_ids"].to(device)
    attention_mask = encodings["attention_mask"].to(device)

    # Free GPU memory
    torch.cuda.empty_cache()

    # Create DataLoader
    dataset = TensorDataset(input_ids, attention_mask)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    predictions = []
    prediction_probs = []

    # Run inference
    with torch.no_grad():
        for batch in data_loader:
            batch_input_ids, batch_attention_mask = batch
            batch_input_ids = batch_input_ids.to(device)
            batch_attention_mask = batch_attention_mask.to(device)

            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)  # Get prediction probabilities

            batch_predictions = torch.argmax(logits, dim=1).cpu().numpy()
            batch_probs = probs.cpu().numpy()

            predictions.extend(batch_predictions)
            prediction_probs.extend(batch_probs)

    # Convert to DataFrame
    pred_df = pd.DataFrame({
        "NCT Number": nct_numbers,
        f"Predicted Label_{text_column}": predictions,
        f"Probability Class 0_{text_column}": [prob[0] for prob in prediction_probs],
        f"Probability Class 1_{text_column}": [prob[1] for prob in prediction_probs]
    })

    # Merge with original DataFrame
    data = pd.merge(data, pred_df, on="NCT Number", how="left")
    
    return data

# Run inference on test data
data = predict(
    data=data,  # Your test dataset
    tokenizer=tokenizer,
    model=model,
    device=device,
    text_column="Study Title",
    max_length=56,
    batch_size=16
)

# Display updated DataFrame
data.head()

,NCT Number,Study Title,Study URL,Acronym,Brief Summary,Study Results,Conditions,Interventions,Primary Outcome Measures,Secondary Outcome Measures,...,First Posted,Results First Posted,Last Update Posted,Locations,Study Documents,Conditions_Category,Outcomes,Predicted Label_Study Title,Probability Class 0_Study Title,Probability Class 1_Study Title
0,NCT03162666,Patient Outcomes Using an Expandable Spacer,https://clinicaltrials.gov/study/NCT03162666,NaN,This study is a post-market clinical follow-up...,NO,Degenerative Disc Disease,DEVICE: ALTERA,"Change in Radiographic Analysis, Global and Se...","Complications, Surgical or device related comp...",...,5/22/17,NaN,8/23/21,"Rush University Medical Center, Chicago, Illin...",NaN,Non-Oncology,"Change in Radiographic Analysis, Global and Se...",0,0.978512,0.021488
1,NCT04312048,the Effect of Isosorbide Mononitrate in Reduci...,https://clinicaltrials.gov/study/NCT04312048,NaN,The aim of the study is to evaluate the effica...,NO,IUD Insertion Pain,DRUG: Isosorbide mononitrate|DRUG: placebo,"pain during IUD insertion, intensity of patien...","duration of IUD insertion, duration of IUD ins...",...,3/18/20,NaN,1/20/21,"Ahmed Samy, Giza, 11231, Egypt",NaN,Non-Oncology,"pain during IUD insertion, intensity of patien...",1,0.001262,0.998738
2,NCT03144778,Durvalumab With or Without Tremelimumab in Tre...,https://clinicaltrials.gov/study/NCT03144778,NaN,This phase I trial studies how well durvalumab...,NO,Stage II Oropharyngeal Squamous Cell Carcinoma...,BIOLOGICAL: Durvalumab|BIOLOGICAL: Tremelimumab,"Change of CD8+ tumor infiltrating lymphocytes,...",Incidence of adverse events and serious advers...,...,05-09-17,NaN,3/18/21,"M D Anderson Cancer Center, Houston, Texas, 77...",NaN,Oncology,"Change of CD8+ tumor infiltrating lymphocytes,...",0,0.952378,0.047622
3,NCT01592721,Radiation and Cetuximab Plus Intratumoral EGFR...,https://clinicaltrials.gov/study/NCT01592721,NaN,The incorporation of novel targeted therapies ...,YES,Squamous Cell Carcinoma|Head and Neck Cancer,BIOLOGICAL: EGFR Antisense DNA,"Toxicity Rate, This is a 2-stage clinical tria...","Tumor Response, Clinical secondary endpoints i...",...,05-07-12,9/21/22,9/21/22,"University of Pittsburgh, Pittsburgh, Pennsylv...","Study Protocol and Statistical Analysis Plan, ...",Oncology,"Toxicity Rate, This is a 2-stage clinical tria...",0,0.613021,0.386979
4,NCT04253613,Laser Biostimulation in Periodontal Treatment,https://clinicaltrials.gov/study/NCT04253613,NaN,"The use of nonsurgical periodontal treatment, ...",NO,Periodontal Inflammation|Periodontal Diseases,NaN,"IL-1β level in GCF, IL-1β is a cytokine presen...",NaN,...,02-05-20,NaN,07-06-22,NaN,NaN,Non-Oncology,"IL-1β level in GCF, IL-1β is a cytokine presen...",1,0.002673,0.997327


In [177]:
model_path = folder_path+ "ClinicalBioBertFinal/brief_summary_fine_tuned_clinicalbiobert_v3"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Set Device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # Set model to evaluation mode

# Run inference on test data
data = predict(
    data=data,  # Your test dataset
    tokenizer=tokenizer,
    model=model,
    device=device,
    text_column="Brief Summary",
    max_length=128,
    batch_size=16
)

# Display updated DataFrame
data.head()

,NCT Number,Study Title,Study URL,Acronym,Brief Summary,Study Results,Conditions,Interventions,Primary Outcome Measures,Secondary Outcome Measures,...,Locations,Study Documents,Conditions_Category,Outcomes,Predicted Label_Study Title,Probability Class 0_Study Title,Probability Class 1_Study Title,Predicted Label_Brief Summary,Probability Class 0_Brief Summary,Probability Class 1_Brief Summary
0,NCT03162666,Patient Outcomes Using an Expandable Spacer,https://clinicaltrials.gov/study/NCT03162666,NaN,This study is a post-market clinical follow-up...,NO,Degenerative Disc Disease,DEVICE: ALTERA,"Change in Radiographic Analysis, Global and Se...","Complications, Surgical or device related comp...",...,"Rush University Medical Center, Chicago, Illin...",NaN,Non-Oncology,"Change in Radiographic Analysis, Global and Se...",0,0.978512,0.021488,1,0.070896,0.929104
1,NCT04312048,the Effect of Isosorbide Mononitrate in Reduci...,https://clinicaltrials.gov/study/NCT04312048,NaN,The aim of the study is to evaluate the effica...,NO,IUD Insertion Pain,DRUG: Isosorbide mononitrate|DRUG: placebo,"pain during IUD insertion, intensity of patien...","duration of IUD insertion, duration of IUD ins...",...,"Ahmed Samy, Giza, 11231, Egypt",NaN,Non-Oncology,"pain during IUD insertion, intensity of patien...",1,0.001262,0.998738,1,0.000164,0.999836
2,NCT03144778,Durvalumab With or Without Tremelimumab in Tre...,https://clinicaltrials.gov/study/NCT03144778,NaN,This phase I trial studies how well durvalumab...,NO,Stage II Oropharyngeal Squamous Cell Carcinoma...,BIOLOGICAL: Durvalumab|BIOLOGICAL: Tremelimumab,"Change of CD8+ tumor infiltrating lymphocytes,...",Incidence of adverse events and serious advers...,...,"M D Anderson Cancer Center, Houston, Texas, 77...",NaN,Oncology,"Change of CD8+ tumor infiltrating lymphocytes,...",0,0.952378,0.047622,0,0.810098,0.189902
3,NCT01592721,Radiation and Cetuximab Plus Intratumoral EGFR...,https://clinicaltrials.gov/study/NCT01592721,NaN,The incorporation of novel targeted therapies ...,YES,Squamous Cell Carcinoma|Head and Neck Cancer,BIOLOGICAL: EGFR Antisense DNA,"Toxicity Rate, This is a 2-stage clinical tria...","Tumor Response, Clinical secondary endpoints i...",...,"University of Pittsburgh, Pittsburgh, Pennsylv...","Study Protocol and Statistical Analysis Plan, ...",Oncology,"Toxicity Rate, This is a 2-stage clinical tria...",0,0.613021,0.386979,0,0.938475,0.061525
4,NCT04253613,Laser Biostimulation in Periodontal Treatment,https://clinicaltrials.gov/study/NCT04253613,NaN,"The use of nonsurgical periodontal treatment, ...",NO,Periodontal Inflammation|Periodontal Diseases,NaN,"IL-1β level in GCF, IL-1β is a cytokine presen...",NaN,...,NaN,NaN,Non-Oncology,"IL-1β level in GCF, IL-1β is a cytokine presen...",1,0.002673,0.997327,1,0.000860,0.999140


In [178]:
model_path = folder_path+ "ClinicalBioBertFinal/Conditions_fine_tuned_clinicalbiobert_v3"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Set Device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # Set model to evaluation mode

# Run inference on test data
data = predict(
    data=data,  # Your test dataset
    tokenizer=tokenizer,
    model=model,
    device=device,
    text_column="Conditions",
    max_length=128,
    batch_size=16
)

# Display updated DataFrame
data.head()

,NCT Number,Study Title,Study URL,Acronym,Brief Summary,Study Results,Conditions,Interventions,Primary Outcome Measures,Secondary Outcome Measures,...,Outcomes,Predicted Label_Study Title,Probability Class 0_Study Title,Probability Class 1_Study Title,Predicted Label_Brief Summary,Probability Class 0_Brief Summary,Probability Class 1_Brief Summary,Predicted Label_Conditions,Probability Class 0_Conditions,Probability Class 1_Conditions
0,NCT03162666,Patient Outcomes Using an Expandable Spacer,https://clinicaltrials.gov/study/NCT03162666,NaN,This study is a post-market clinical follow-up...,NO,Degenerative Disc Disease,DEVICE: ALTERA,"Change in Radiographic Analysis, Global and Se...","Complications, Surgical or device related comp...",...,"Change in Radiographic Analysis, Global and Se...",0,0.978512,0.021488,1,0.070896,0.929104,0,0.749659,0.250341
1,NCT04312048,the Effect of Isosorbide Mononitrate in Reduci...,https://clinicaltrials.gov/study/NCT04312048,NaN,The aim of the study is to evaluate the effica...,NO,IUD Insertion Pain,DRUG: Isosorbide mononitrate|DRUG: placebo,"pain during IUD insertion, intensity of patien...","duration of IUD insertion, duration of IUD ins...",...,"pain during IUD insertion, intensity of patien...",1,0.001262,0.998738,1,0.000164,0.999836,1,0.404198,0.595802
2,NCT03144778,Durvalumab With or Without Tremelimumab in Tre...,https://clinicaltrials.gov/study/NCT03144778,NaN,This phase I trial studies how well durvalumab...,NO,Stage II Oropharyngeal Squamous Cell Carcinoma...,BIOLOGICAL: Durvalumab|BIOLOGICAL: Tremelimumab,"Change of CD8+ tumor infiltrating lymphocytes,...",Incidence of adverse events and serious advers...,...,"Change of CD8+ tumor infiltrating lymphocytes,...",0,0.952378,0.047622,0,0.810098,0.189902,0,0.769768,0.230232
3,NCT01592721,Radiation and Cetuximab Plus Intratumoral EGFR...,https://clinicaltrials.gov/study/NCT01592721,NaN,The incorporation of novel targeted therapies ...,YES,Squamous Cell Carcinoma|Head and Neck Cancer,BIOLOGICAL: EGFR Antisense DNA,"Toxicity Rate, This is a 2-stage clinical tria...","Tumor Response, Clinical secondary endpoints i...",...,"Toxicity Rate, This is a 2-stage clinical tria...",0,0.613021,0.386979,0,0.938475,0.061525,0,0.764094,0.235906
4,NCT04253613,Laser Biostimulation in Periodontal Treatment,https://clinicaltrials.gov/study/NCT04253613,NaN,"The use of nonsurgical periodontal treatment, ...",NO,Periodontal Inflammation|Periodontal Diseases,NaN,"IL-1β level in GCF, IL-1β is a cytokine presen...",NaN,...,"IL-1β level in GCF, IL-1β is a cytokine presen...",1,0.002673,0.997327,1,0.000860,0.999140,1,0.145164,0.854836


In [179]:
model_path = folder_path+ "ClinicalBioBertFinal/Outcomes_fine_tuned_clinicalbiobert_v3"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Set Device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # Set model to evaluation mode

# Run inference on test data
data = predict(
    data=data,  # Your test dataset
    tokenizer=tokenizer,
    model=model,
    device=device,
    text_column="Outcomes",
    max_length=128,
    batch_size=16
)

# Display updated DataFrame
data.head()

,NCT Number,Study Title,Study URL,Acronym,Brief Summary,Study Results,Conditions,Interventions,Primary Outcome Measures,Secondary Outcome Measures,...,Probability Class 1_Study Title,Predicted Label_Brief Summary,Probability Class 0_Brief Summary,Probability Class 1_Brief Summary,Predicted Label_Conditions,Probability Class 0_Conditions,Probability Class 1_Conditions,Predicted Label_Outcomes,Probability Class 0_Outcomes,Probability Class 1_Outcomes
0,NCT03162666,Patient Outcomes Using an Expandable Spacer,https://clinicaltrials.gov/study/NCT03162666,NaN,This study is a post-market clinical follow-up...,NO,Degenerative Disc Disease,DEVICE: ALTERA,"Change in Radiographic Analysis, Global and Se...","Complications, Surgical or device related comp...",...,0.021488,1,0.070896,0.929104,0,0.749659,0.250341,0,0.982102,0.017899
1,NCT04312048,the Effect of Isosorbide Mononitrate in Reduci...,https://clinicaltrials.gov/study/NCT04312048,NaN,The aim of the study is to evaluate the effica...,NO,IUD Insertion Pain,DRUG: Isosorbide mononitrate|DRUG: placebo,"pain during IUD insertion, intensity of patien...","duration of IUD insertion, duration of IUD ins...",...,0.998738,1,0.000164,0.999836,1,0.404198,0.595802,1,0.000473,0.999527
2,NCT03144778,Durvalumab With or Without Tremelimumab in Tre...,https://clinicaltrials.gov/study/NCT03144778,NaN,This phase I trial studies how well durvalumab...,NO,Stage II Oropharyngeal Squamous Cell Carcinoma...,BIOLOGICAL: Durvalumab|BIOLOGICAL: Tremelimumab,"Change of CD8+ tumor infiltrating lymphocytes,...",Incidence of adverse events and serious advers...,...,0.047622,0,0.810098,0.189902,0,0.769768,0.230232,0,0.941957,0.058043
3,NCT01592721,Radiation and Cetuximab Plus Intratumoral EGFR...,https://clinicaltrials.gov/study/NCT01592721,NaN,The incorporation of novel targeted therapies ...,YES,Squamous Cell Carcinoma|Head and Neck Cancer,BIOLOGICAL: EGFR Antisense DNA,"Toxicity Rate, This is a 2-stage clinical tria...","Tumor Response, Clinical secondary endpoints i...",...,0.386979,0,0.938475,0.061525,0,0.764094,0.235906,0,0.765026,0.234974
4,NCT04253613,Laser Biostimulation in Periodontal Treatment,https://clinicaltrials.gov/study/NCT04253613,NaN,"The use of nonsurgical periodontal treatment, ...",NO,Periodontal Inflammation|Periodontal Diseases,NaN,"IL-1β level in GCF, IL-1β is a cytokine presen...",NaN,...,0.997327,1,0.000860,0.999140,1,0.145164,0.854836,1,0.000651,0.999349


In [180]:
list(data.columns)

['NCT Number',
 'Study Title',
 'Study URL',
 'Acronym',
 'Brief Summary',
 'Study Results',
 'Conditions',
 'Interventions',
 'Primary Outcome Measures',
 'Secondary Outcome Measures',
 'Other Outcome Measures',
 'Sponsor',
 'Collaborators',
 'Sex',
 'Age',
 'Phases',
 'Enrollment',
 'Funder Type',
 'Study Type',
 'Study Design',
 'Other IDs',
 'Start Date',
 'Primary Completion Date',
 'Completion Date',
 'First Posted',
 'Results First Posted',
 'Last Update Posted',
 'Locations',
 'Study Documents',
 'Conditions_Category',
 'Outcomes',
 'Predicted Label_Study Title',
 'Probability Class 0_Study Title',
 'Probability Class 1_Study Title',
 'Predicted Label_Brief Summary',
 'Probability Class 0_Brief Summary',
 'Probability Class 1_Brief Summary',
 'Predicted Label_Conditions',
 'Probability Class 0_Conditions',
 'Probability Class 1_Conditions',
 'Predicted Label_Outcomes',
 'Probability Class 0_Outcomes',
 'Probability Class 1_Outcomes']

In [181]:
data = data.drop(columns=['Study URL', 'Acronym', 'Study Results', 'Phases', 'Enrollment', 'Other IDs', 'Primary Completion Date', 'Completion Date', 'First Posted', 'Results First Posted', 'Last Update Posted', 'Study Documents', 'Primary Outcome Measures', 'Secondary Outcome Measures', 'Other Outcome Measures'], errors='ignore')
data.head()

,NCT Number,Study Title,Brief Summary,Conditions,Interventions,Sponsor,Collaborators,Sex,Age,Funder Type,...,Probability Class 1_Study Title,Predicted Label_Brief Summary,Probability Class 0_Brief Summary,Probability Class 1_Brief Summary,Predicted Label_Conditions,Probability Class 0_Conditions,Probability Class 1_Conditions,Predicted Label_Outcomes,Probability Class 0_Outcomes,Probability Class 1_Outcomes
0,NCT03162666,Patient Outcomes Using an Expandable Spacer,This study is a post-market clinical follow-up...,Degenerative Disc Disease,DEVICE: ALTERA,Globus Medical Inc,NaN,ALL,"ADULT, OLDER_ADULT",INDUSTRY,...,0.021488,1,0.070896,0.929104,0,0.749659,0.250341,0,0.982102,0.017899
1,NCT04312048,the Effect of Isosorbide Mononitrate in Reduci...,The aim of the study is to evaluate the effica...,IUD Insertion Pain,DRUG: Isosorbide mononitrate|DRUG: placebo,Cairo University,NaN,FEMALE,"CHILD, ADULT, OLDER_ADULT",OTHER,...,0.998738,1,0.000164,0.999836,1,0.404198,0.595802,1,0.000473,0.999527
2,NCT03144778,Durvalumab With or Without Tremelimumab in Tre...,This phase I trial studies how well durvalumab...,Stage II Oropharyngeal Squamous Cell Carcinoma...,BIOLOGICAL: Durvalumab|BIOLOGICAL: Tremelimumab,M.D. Anderson Cancer Center,National Cancer Institute (NCI),ALL,"ADULT, OLDER_ADULT",OTHER,...,0.047622,0,0.810098,0.189902,0,0.769768,0.230232,0,0.941957,0.058043
3,NCT01592721,Radiation and Cetuximab Plus Intratumoral EGFR...,The incorporation of novel targeted therapies ...,Squamous Cell Carcinoma|Head and Neck Cancer,BIOLOGICAL: EGFR Antisense DNA,The University of Texas Health Science Center ...,University of Pittsburgh,ALL,"ADULT, OLDER_ADULT",OTHER,...,0.386979,0,0.938475,0.061525,0,0.764094,0.235906,0,0.765026,0.234974
4,NCT04253613,Laser Biostimulation in Periodontal Treatment,"The use of nonsurgical periodontal treatment, ...",Periodontal Inflammation|Periodontal Diseases,NaN,Ondokuz Mayıs University,NaN,MALE,"ADULT, OLDER_ADULT",OTHER,...,0.997327,1,0.000860,0.999140,1,0.145164,0.854836,1,0.000651,0.999349


In [182]:
# Ensure Start Date is in datetime format
data['Start Date'] = pd.to_datetime(data['Start Date'], errors='coerce')

# Extract month and quarter, handling missing values
data['Start Month'] = data['Start Date'].dt.month.fillna(-1).astype(int).astype('category')
data['Start Quarter'] = data['Start Date'].dt.quarter.fillna(-1).astype(int).astype('category')

# Verify the changes
data[['Start Date', 'Start Month', 'Start Quarter']].head()

C:\Users\mogal\AppData\Local\Temp\ipykernel_42584\4196633104.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['Start Date'] = pd.to_datetime(data['Start Date'], errors='coerce')


,Start Date,Start Month,Start Quarter
0,2017-02-28,2,1
1,2020-04-15,4,2
2,2017-07-12,7,3
3,2013-04-01,4,2
4,2013-01-15,1,1


In [183]:
# Count number of collaborators per trial
data['Num_Collaborators'] = data['Collaborators'].apply(lambda x: len(str(x).split('|')) if pd.notna(x) else 0)
# Count number of collaborators per trial
data['Num_Collaborators'] = data['Collaborators'].apply(lambda x: len(str(x).split('|')) if pd.notna(x) else 0)
# Expanded keyword-based classification
def refine_collaborator_type(collaborator):
    if pd.isna(collaborator) or collaborator.strip() == "":
        return "Unknown"

    collaborator = collaborator.lower()

    # Pharmaceutical/Biotech Companies
    if any(x in collaborator for x in ["pharma", "biotech", "therapeutics", "biosciences", "genomics", "biopharma", "life sciences", "vaccines", "therapeutics"]):
        return "Pharmaceutical/Biotech"

    # Medical Device Companies
    elif any(x in collaborator for x in ["medtech", "medical devices", "diagnostics", "boston scientific", "medtronic", "abbott", "siemens healthineers", "imaging", "st. jude medical", "stryker"]):
        return "Medical Device Company"

    # Universities & Academic Institutions
    elif any(x in collaborator for x in ["university", "college", "academy", "school of medicine", "medical school"]):
        return "University"

    # Hospitals & Medical Centers
    elif any(x in collaborator for x in ["hospital", "medical center", "health system", "clinic", "health network", "cancer center", "teaching hospital"]):
        return "Hospital/Medical Center"

    # Academic Research Centers
    elif any(x in collaborator for x in ["institute of technology", "research institute", "academic research", "broad institute", "howard hughes", "karolinska institute"]):
        return "Academic Research Institute"

    # Non-Academic Research Centers
    elif any(x in collaborator for x in ["national laboratory", "max planck", "clinical research", "research foundation"]):
        return "Non-Academic Research Institute"

    # Government & Public Health Agencies
    elif any(x in collaborator for x in ["national", "federal", "ministry of health", "nih", "fda", "cdc", "who", "government", "department of health", "european medicines agency", "health canada"]):
        return "Government"

    # Military & Defense Medical Research
    elif any(x in collaborator for x in ["military", "army", "navy", "air force", "veterans affairs", "dod", "defense", "va medical", "darpa", "us army medical research"]):
        return "Military/Defense Medical Research"

    # Non-Profit Organizations & NGOs
    elif any(x in collaborator for x in ["non-profit", "ngo", "humanitarian", "relief fund", "patient advocacy", "community group", "alzheimer's association", "parkinson's foundation"]):
        return "Non-Profit / Patient Advocacy"

    # Foundations (Funding Agencies)
    elif any(x in collaborator for x in ["foundation", "charitable trust", "funding agency", "gates foundation", "michael j. fox foundation"]):
        return "Foundation (Funding Organization)"

    # Contract Research Organizations (CROs)
    elif any(x in collaborator for x in ["quintiles", "iqvia", "parexel", "covance", "medpace", "ppd", "icon", "charles river", "syneos", "contract research"]):
        return "Contract Research Organization (CRO)"

    # Healthcare Consortia & Alliances
    elif any(x in collaborator for x in ["consortium", "partnership", "collaborative", "alliance", "global initiative"]):
        return "Healthcare Consortium/Alliance"

    # If no match, return Other
    else:
        return "Other"

# Apply classification
data['Collaborator_Type'] = data['Collaborators'].apply(refine_collaborator_type)
data.head()

,NCT Number,Study Title,Brief Summary,Conditions,Interventions,Sponsor,Collaborators,Sex,Age,Funder Type,...,Predicted Label_Conditions,Probability Class 0_Conditions,Probability Class 1_Conditions,Predicted Label_Outcomes,Probability Class 0_Outcomes,Probability Class 1_Outcomes,Start Month,Start Quarter,Num_Collaborators,Collaborator_Type
0,NCT03162666,Patient Outcomes Using an Expandable Spacer,This study is a post-market clinical follow-up...,Degenerative Disc Disease,DEVICE: ALTERA,Globus Medical Inc,NaN,ALL,"ADULT, OLDER_ADULT",INDUSTRY,...,0,0.749659,0.250341,0,0.982102,0.017899,2,1,0,Unknown
1,NCT04312048,the Effect of Isosorbide Mononitrate in Reduci...,The aim of the study is to evaluate the effica...,IUD Insertion Pain,DRUG: Isosorbide mononitrate|DRUG: placebo,Cairo University,NaN,FEMALE,"CHILD, ADULT, OLDER_ADULT",OTHER,...,1,0.404198,0.595802,1,0.000473,0.999527,4,2,0,Unknown
2,NCT03144778,Durvalumab With or Without Tremelimumab in Tre...,This phase I trial studies how well durvalumab...,Stage II Oropharyngeal Squamous Cell Carcinoma...,BIOLOGICAL: Durvalumab|BIOLOGICAL: Tremelimumab,M.D. Anderson Cancer Center,National Cancer Institute (NCI),ALL,"ADULT, OLDER_ADULT",OTHER,...,0,0.769768,0.230232,0,0.941957,0.058043,7,3,1,Government
3,NCT01592721,Radiation and Cetuximab Plus Intratumoral EGFR...,The incorporation of novel targeted therapies ...,Squamous Cell Carcinoma|Head and Neck Cancer,BIOLOGICAL: EGFR Antisense DNA,The University of Texas Health Science Center ...,University of Pittsburgh,ALL,"ADULT, OLDER_ADULT",OTHER,...,0,0.764094,0.235906,0,0.765026,0.234974,4,2,1,University
4,NCT04253613,Laser Biostimulation in Periodontal Treatment,"The use of nonsurgical periodontal treatment, ...",Periodontal Inflammation|Periodontal Diseases,NaN,Ondokuz Mayıs University,NaN,MALE,"ADULT, OLDER_ADULT",OTHER,...,1,0.145164,0.854836,1,0.000651,0.999349,1,1,0,Unknown


In [184]:
data['Sex'] = data['Sex'].fillna('ALL')

In [185]:
data = data.drop(columns=['Collaborators'])

In [186]:
data = data.dropna(subset=['Sponsor','Age','Funder Type','Study Type','Study Design'])

In [187]:
# Lists of columns
columns_mode = ['Age', 'Sex']
columns_unknown = ['Funder Type', 'Study Type', 'Study Design', 'Outcomes']

# Fill numerical columns with median
for col in columns_mode:
    mode_value = data[col].mode()
    data[col].fillna(mode_value, inplace=True)

# Fill categorical columns with 'unknown'
for col in columns_unknown:
    data[col].fillna('Unknown', inplace=True)

C:\Users\mogal\AppData\Local\Temp\ipykernel_42584\3234544013.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(mode_value, inplace=True)
C:\Users\mogal\AppData\Local\Temp\ipykernel_42584\3234544013.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example,

In [188]:

# Define a function to classify conditions with detailed categories
def classify_condition(condition):
    if pd.isnull(condition):
        return 'Unknown'

    condition_lower = condition.lower()

    # Cancer-related conditions
    if any(keyword in condition_lower for keyword in ['cancer', 'tumor', 'carcinoma', 'lymphoma', 'leukemia', 'melanoma']):
        return 'Cancer'

    # Cardiovascular diseases
    if any(keyword in condition_lower for keyword in ['heart', 'cardio', 'vascular', 'stroke', 'artery', 'hypertension', 'atrial']):
        return 'Cardiovascular'

    # Neurological disorders
    if any(keyword in condition_lower for keyword in ['brain', 'neuro', 'alzheimer', 'parkinson', 'epilepsy', 'multiple sclerosis']):
        return 'Neurological'

    # Infectious diseases
    if any(keyword in condition_lower for keyword in ['infection', 'virus', 'bacterial', 'hiv', 'malaria', 'covid', 'influenza']):
        return 'Infectious Diseases'

    # Endocrine and metabolic disorders
    if any(keyword in condition_lower for keyword in ['diabetes', 'thyroid', 'obesity', 'metabolic', 'insulin']):
        return 'Endocrine/Metabolic'

    # Respiratory conditions
    if any(keyword in condition_lower for keyword in ['asthma', 'lung', 'copd', 'pulmonary']):
        return 'Respiratory'

    # Gastrointestinal & Hepatic diseases
    if any(keyword in condition_lower for keyword in ['liver', 'hepatic', 'pancreatic', 'colorectal', 'gastro']):
        return 'Gastrointestinal & Hepatic'

    # Autoimmune and inflammatory conditions
    if any(keyword in condition_lower for keyword in ['arthritis', 'rheumatoid', 'psoriasis', 'inflammatory', 'lupus', 'crohn']):
        return 'Autoimmune & Inflammatory'

    # Musculoskeletal disorders
    if any(keyword in condition_lower for keyword in ['osteoporosis', 'osteoarthritis', 'fracture', 'spine', 'back pain', 'knee']):
        return 'Musculoskeletal'

    # Psychiatric & behavioral disorders
    if any(keyword in condition_lower for keyword in ['schizophrenia', 'bipolar', 'depression', 'psychiatric', 'mental health']):
        return 'Psychiatric & Behavioral'

    # Reproductive & urological disorders
    if any(keyword in condition_lower for keyword in ['infertility', 'prostate', 'kidney', 'renal', 'bladder', 'reproductive']):
        return 'Reproductive & Urological'

    # General health & other
    if any(keyword in condition_lower for keyword in ['healthy', 'general health', 'volunteers']):
        return 'General Health & Others'

    # Default to "Other" for unspecified conditions
    return 'Other'

# Apply the improved classification function
data['Condition_Category_old'] = data['Conditions'].apply(classify_condition)

# Verify results
data[['Conditions', 'Condition_Category_old']].head(20)

,Conditions,Condition_Category_old
0,Degenerative Disc Disease,Other
1,IUD Insertion Pain,Other
2,Stage II Oropharyngeal Squamous Cell Carcinoma...,Cancer
3,Squamous Cell Carcinoma|Head and Neck Cancer,Cancer
4,Periodontal Inflammation|Periodontal Diseases,Other
5,Peripheral Artery Disease,Cardiovascular
6,Cardiovascular Risk Factor|Heart Diseases|Infl...,Cardiovascular
7,Healthy,General Health & Others
8,Parkinson's Disease|With Inclusion Criteria fo...,Neurological
9,Urinary Tract Infections,Infectious Diseases


In [189]:
data.drop(columns=['Conditions'], inplace=True)

In [190]:
# Define a function to classify sponsors
def classify_sponsor(sponsor):
    if pd.isnull(sponsor):
        return 'Unknown'
    sponsor_lower = sponsor.lower()

    # University
    if 'university' in sponsor_lower or 'school of medicine' in sponsor_lower:
        return 'University'

    # Hospital or Clinic
    if 'hospital' in sponsor_lower or 'clinic' in sponsor_lower or 'center' in sponsor_lower:
        return 'Hospital/Clinic'

    # Company
    company_keywords = ['pharmaceuticals', 'biopharma', 'biosciences', 'llc', 'inc',
                         'company', 'a/s', 'ltd', 'plc', 'corporation']
    if any(keyword in sponsor_lower for keyword in company_keywords):
        return 'Company'

    # Government Institute
    gov_keywords = ['national', 'institute', 'office of research', 'va', 'nci', 'niaid']
    if any(keyword in sponsor_lower for keyword in gov_keywords):
        return 'Government Institute'

    # Other
    return 'Other'

# Apply the classification function to the Sponsor column
data['Sponsor Type'] = data['Sponsor'].apply(classify_sponsor)
data.drop(columns=['Sponsor'], inplace=True)
# Verify the distribution of Sponsor Type
print(data['Sponsor Type'].value_counts())

Sponsor Type
University              41
Other                   25
Company                 13
Hospital/Clinic         13
Government Institute     8
Name: count, dtype: int64


In [191]:
# Define the unique age categories
age_categories = ['CHILD', 'ADULT', 'OLDER_ADULT']

# Create binary columns for each age group
for category in age_categories:
    data[category] = data['Age'].apply(lambda x: 1 if isinstance(x, str) and category in x else 0)

# Drop the original 'Age' column
data.drop(columns=['Age'], inplace=True)

# Display the modified DataFrame
data.head()

,NCT Number,Study Title,Brief Summary,Interventions,Sex,Funder Type,Study Type,Study Design,Start Date,Locations,...,Probability Class 1_Outcomes,Start Month,Start Quarter,Num_Collaborators,Collaborator_Type,Condition_Category_old,Sponsor Type,CHILD,ADULT,OLDER_ADULT
0,NCT03162666,Patient Outcomes Using an Expandable Spacer,This study is a post-market clinical follow-up...,DEVICE: ALTERA,ALL,INDUSTRY,OBSERVATIONAL,Observational Model: |Time Perspective: p,2017-02-28,"Rush University Medical Center, Chicago, Illin...",...,0.017899,2,1,0,Unknown,Other,Company,0,1,1
1,NCT04312048,the Effect of Isosorbide Mononitrate in Reduci...,The aim of the study is to evaluate the effica...,DRUG: Isosorbide mononitrate|DRUG: placebo,FEMALE,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2020-04-15,"Ahmed Samy, Giza, 11231, Egypt",...,0.999527,4,2,0,Unknown,Other,University,1,1,1
2,NCT03144778,Durvalumab With or Without Tremelimumab in Tre...,This phase I trial studies how well durvalumab...,BIOLOGICAL: Durvalumab|BIOLOGICAL: Tremelimumab,ALL,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2017-07-12,"M D Anderson Cancer Center, Houston, Texas, 77...",...,0.058043,7,3,1,Government,Cancer,Hospital/Clinic,0,1,1
3,NCT01592721,Radiation and Cetuximab Plus Intratumoral EGFR...,The incorporation of novel targeted therapies ...,BIOLOGICAL: EGFR Antisense DNA,ALL,OTHER,INTERVENTIONAL,Allocation: NA|Intervention Model: SINGLE_GROU...,2013-04-01,"University of Pittsburgh, Pittsburgh, Pennsylv...",...,0.234974,4,2,1,University,Cancer,University,0,1,1
4,NCT04253613,Laser Biostimulation in Periodontal Treatment,"The use of nonsurgical periodontal treatment, ...",NaN,MALE,OTHER,OBSERVATIONAL,Observational Model: |Time Perspective: p,2013-01-15,NaN,...,0.999349,1,1,0,Unknown,Other,University,0,1,1


In [192]:
# Function to extract attribute values
def extract_attribute(study_design, attribute):
    if pd.isnull(study_design):
        return None
    for part in study_design.split('|'):
        if attribute in part:
            return part.split(':')[1].strip() if ':' in part else None
    return None

# Extract key features
data['Allocation'] = data['Study Design'].apply(lambda x: extract_attribute(x, 'Allocation'))
data['Intervention Model'] = data['Study Design'].apply(lambda x: extract_attribute(x, 'Intervention Model'))
data['Masking'] = data['Study Design'].apply(lambda x: extract_attribute(x, 'Masking'))
data['Primary Purpose'] = data['Study Design'].apply(lambda x: extract_attribute(x, 'Primary Purpose'))
data['Observational Model'] = data['Study Design'].apply(lambda x: extract_attribute(x, 'Observational Model'))
data['Time Perspective'] = data['Study Design'].apply(lambda x: extract_attribute(x, 'Time Perspective'))

# Extract Masking Level
def extract_masking_level(masking):
    if pd.isnull(masking):
        return 'NONE'
    if 'QUADRUPLE' in masking:
        return 'QUADRUPLE'
    if 'TRIPLE' in masking:
        return 'TRIPLE'
    if 'DOUBLE' in masking:
        return 'DOUBLE'
    if 'SINGLE' in masking:
        return 'SINGLE'
    return 'NONE'

data['Masking Level'] = data['Masking'].apply(extract_masking_level)

# Extract Masking Details
def extract_masking_details(masking):
    if pd.isnull(masking):
        return 'NONE'
    if '(' in masking:
        return masking.split('(')[1].strip(')')
    return 'NONE'

data['Masking Details'] = data['Masking'].apply(extract_masking_details)

# Handle missing values by filling with 'Unknown'
data['Allocation'].fillna('Unknown', inplace=True)
data['Intervention Model'].fillna('Unknown', inplace=True)
data['Primary Purpose'].fillna('Unknown', inplace=True)
data['Observational Model'].fillna('Unknown', inplace=True)
data['Time Perspective'].fillna('Unknown', inplace=True)

# One-hot encode categorical features
data = pd.get_dummies(
    data,
    columns=['Allocation', 'Intervention Model', 'Masking Level', 'Primary Purpose']
)

# Drop original 'Study Design' column
# data.drop(columns=['Study Design'], inplace=True)
data.head()

C:\Users\mogal\AppData\Local\Temp\ipykernel_42584\493715576.py:45: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Allocation'].fillna('Unknown', inplace=True)
C:\Users\mogal\AppData\Local\Temp\ipykernel_42584\493715576.py:46: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

,NCT Number,Study Title,Brief Summary,Interventions,Sex,Funder Type,Study Type,Study Design,Start Date,Locations,...,Primary Purpose_,Primary Purpose_BASIC_SCIENCE,Primary Purpose_DIAGNOSTIC,Primary Purpose_HEALTH_SERVICES_RESEARCH,Primary Purpose_OTHER,Primary Purpose_PREVENTION,Primary Purpose_SCREENING,Primary Purpose_SUPPORTIVE_CARE,Primary Purpose_TREATMENT,Primary Purpose_Unknown
0,NCT03162666,Patient Outcomes Using an Expandable Spacer,This study is a post-market clinical follow-up...,DEVICE: ALTERA,ALL,INDUSTRY,OBSERVATIONAL,Observational Model: |Time Perspective: p,2017-02-28,"Rush University Medical Center, Chicago, Illin...",...,False,False,False,False,False,False,False,False,False,True
1,NCT04312048,the Effect of Isosorbide Mononitrate in Reduci...,The aim of the study is to evaluate the effica...,DRUG: Isosorbide mononitrate|DRUG: placebo,FEMALE,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2020-04-15,"Ahmed Samy, Giza, 11231, Egypt",...,False,False,False,False,False,True,False,False,False,False
2,NCT03144778,Durvalumab With or Without Tremelimumab in Tre...,This phase I trial studies how well durvalumab...,BIOLOGICAL: Durvalumab|BIOLOGICAL: Tremelimumab,ALL,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2017-07-12,"M D Anderson Cancer Center, Houston, Texas, 77...",...,False,False,False,False,False,False,False,False,True,False
3,NCT01592721,Radiation and Cetuximab Plus Intratumoral EGFR...,The incorporation of novel targeted therapies ...,BIOLOGICAL: EGFR Antisense DNA,ALL,OTHER,INTERVENTIONAL,Allocation: NA|Intervention Model: SINGLE_GROU...,2013-04-01,"University of Pittsburgh, Pittsburgh, Pennsylv...",...,False,False,False,False,False,False,False,False,True,False
4,NCT04253613,Laser Biostimulation in Periodontal Treatment,"The use of nonsurgical periodontal treatment, ...",NaN,MALE,OTHER,OBSERVATIONAL,Observational Model: |Time Perspective: p,2013-01-15,NaN,...,False,False,False,False,False,False,False,False,False,True


In [193]:
# Fill NaN values with empty strings before concatenation
data['Combined_Text'] = (
    data['Study Title'].fillna('') + ' ' +
    data['Brief Summary'].fillna('') + ' ' +
    data['Outcomes'].fillna('')
)

In [194]:
# Download wordnet and stopwords
nltk.download('wordnet')
nltk.download('omw-1.4')  # Required for wordnet lemmatizer
nltk.download('stopwords')  # Ensure stopwords are available


# Download necessary resources
nltk.download('punkt')  # Tokenizer
nltk.download('averaged_perceptron_tagger')  # POS tagging
nltk.download('wordnet')  # Lemmatization
nltk.download('stopwords')  # Stopwords
nltk.download('punkt_tab')



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mogal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\mogal\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mogal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mogal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mogal\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mogal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_d

True

In [195]:
# List of 195 countries from the dataset
country_list = [
    "India", "China", "United States", "Indonesia", "Pakistan", "Nigeria", "Brazil", "Bangladesh", "Russia",
    "Ethiopia", "Mexico", "Japan", "Egypt", "Philippines", "DR Congo", "Vietnam", "Iran", "Turkey", "Germany",
    "Thailand", "United Kingdom", "Tanzania", "France", "South Africa", "Italy", "Kenya", "Myanmar", "Colombia",
    "Korea", "Sudan", "Uganda", "Spain", "Algeria", "Iraq", "Argentina", "Afghanistan", "Yemen", "Canada",
    "Poland", "Morocco", "Angola", "Ukraine", "Uzbekistan", "Malaysia", "Mozambique", "Ghana", "Peru", "Saudi Arabia",
    "Madagascar", "Côte D'Ivoire", "Nepal", "Cameroon", "Venezuela", "Niger", "Australia", "North Korea", "Syria",
    "Mali", "Burkina Faso", "Sri Lanka", "Malawi", "Zambia", "Kazakhstan", "Chad", "Chile", "Romania", "Somalia",
    "Senegal", "Guatemala", "Netherlands", "Ecuador", "Cambodia", "Zimbabwe", "Guinea", "Benin", "Rwanda", "Burundi",
    "Bolivia", "Tunisia", "South Sudan", "Haiti", "Belgium", "Jordan", "Dominican Republic", "United Arab Emirates",
    "Cuba", "Honduras", "Czech Republic", "Sweden", "Tajikistan", "Papua New Guinea", "Portugal", "Azerbaijan",
    "Greece", "Hungary", "Togo", "Israel", "Austria", "Belarus", "Switzerland", "Sierra Leone", "Laos", "Turkmenistan",
    "Libya", "Kyrgyzstan", "Paraguay", "Nicaragua", "Bulgaria", "Serbia", "El Salvador", "Congo", "Denmark",
    "Singapore", "Lebanon", "Finland", "Liberia", "Norway", "Slovakia", "State of Palestine",
    "Central African Republic", "Oman", "Ireland", "New Zealand", "Mauritania", "Costa Rica", "Kuwait", "Panama",
    "Croatia", "Georgia", "Eritrea", "Mongolia", "Uruguay", "Bosnia and Herzegovina", "Qatar", "Moldova", "Namibia",
    "Armenia", "Lithuania", "Jamaica", "Albania", "Gambia", "Gabon", "Botswana", "Lesotho", "Guinea-Bissau",
    "Slovenia", "Equatorial Guinea", "Latvia", "North Macedonia", "Bahrain", "Trinidad and Tobago", "Timor-Leste",
    "Estonia", "Cyprus", "Mauritius", "Eswatini", "Djibouti", "Fiji", "Comoros", "Guyana", "Solomon Islands",
    "Bhutan", "Luxembourg", "Montenegro", "Suriname", "Malta", "Maldives", "Micronesia", "Cabo Verde", "Brunei",
    "Belize", "Bahamas", "Iceland", "Vanuatu", "Barbados", "Sao Tome & Principe", "Samoa", "Saint Lucia", "Kiribati",
    "Seychelles", "Grenada", "Tonga", "St. Vincent & Grenadines", "Antigua and Barbuda", "Andorra", "Dominica",
    "Saint Kitts & Nevis", "Liechtenstein", "Monaco", "Marshall Islands", "San Marino", "Palau", "Nauru", "Tuvalu",
    "Holy See","Taiwan","Hong Kong","Czechia","Puerto Rico","Palestinian Territory","Swaziland","Faroe Islands","Réunion","Yugoslavia","French Guiana","Lao People's Democratic Republic","The Former Yugoslav Republic of","GSK Investigational Site","Guadeloupe","French Polynesia","Kosovo","Macau","Guam","Greenland","Saint Kitts and Nevis", "New Caledonia", "Northern Mariana Islands", "New Caledonia", "Novartis Investigative Site", "Palestinian Territories", "Cayman Islands"
]


# Function to extract state and country from location text
def extract_state_country(location):
    location = str(location)  # Ensure it's a string
    country = None
    state = None

    # Extract country from location text
    for c in country_list:
        if c in location:
            country = c
            break

    # Extract state (for US locations)
    location_parts = re.split(r'[|,]', location)  # Split by '|' or ','
    for part in location_parts:
        part = part.strip()
    return country

# Apply function to extract state and country
data[['Country']] = data['Locations'].apply(lambda x: pd.Series(extract_state_country(x)))

# Display results
data.head()

,NCT Number,Study Title,Brief Summary,Interventions,Sex,Funder Type,Study Type,Study Design,Start Date,Locations,...,Primary Purpose_DIAGNOSTIC,Primary Purpose_HEALTH_SERVICES_RESEARCH,Primary Purpose_OTHER,Primary Purpose_PREVENTION,Primary Purpose_SCREENING,Primary Purpose_SUPPORTIVE_CARE,Primary Purpose_TREATMENT,Primary Purpose_Unknown,Combined_Text,Country
0,NCT03162666,Patient Outcomes Using an Expandable Spacer,This study is a post-market clinical follow-up...,DEVICE: ALTERA,ALL,INDUSTRY,OBSERVATIONAL,Observational Model: |Time Perspective: p,2017-02-28,"Rush University Medical Center, Chicago, Illin...",...,False,False,False,False,False,False,False,True,Patient Outcomes Using an Expandable Spacer Th...,United States
1,NCT04312048,the Effect of Isosorbide Mononitrate in Reduci...,The aim of the study is to evaluate the effica...,DRUG: Isosorbide mononitrate|DRUG: placebo,FEMALE,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2020-04-15,"Ahmed Samy, Giza, 11231, Egypt",...,False,False,False,True,False,False,False,False,the Effect of Isosorbide Mononitrate in Reduci...,Egypt
2,NCT03144778,Durvalumab With or Without Tremelimumab in Tre...,This phase I trial studies how well durvalumab...,BIOLOGICAL: Durvalumab|BIOLOGICAL: Tremelimumab,ALL,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2017-07-12,"M D Anderson Cancer Center, Houston, Texas, 77...",...,False,False,False,False,False,False,True,False,Durvalumab With or Without Tremelimumab in Tre...,United States
3,NCT01592721,Radiation and Cetuximab Plus Intratumoral EGFR...,The incorporation of novel targeted therapies ...,BIOLOGICAL: EGFR Antisense DNA,ALL,OTHER,INTERVENTIONAL,Allocation: NA|Intervention Model: SINGLE_GROU...,2013-04-01,"University of Pittsburgh, Pittsburgh, Pennsylv...",...,False,False,False,False,False,False,True,False,Radiation and Cetuximab Plus Intratumoral EGFR...,United States
4,NCT04253613,Laser Biostimulation in Periodontal Treatment,"The use of nonsurgical periodontal treatment, ...",NaN,MALE,OTHER,OBSERVATIONAL,Observational Model: |Time Perspective: p,2013-01-15,NaN,...,False,False,False,False,False,False,False,True,Laser Biostimulation in Periodontal Treatment ...,NaN


In [196]:
data.Country.fillna('Unknown', inplace=True)

C:\Users\mogal\AppData\Local\Temp\ipykernel_42584\3965354434.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data.Country.fillna('Unknown', inplace=True)


In [197]:
developed = [
    "United States", "Canada", "Germany", "Japan", "United Kingdom", "France", "Switzerland",
    "Netherlands", "Sweden", "Denmark", "Norway", "Finland", "Australia", "New Zealand", "Belgium",
    "Austria", "Ireland", "Singapore", "Hong Kong", "Taiwan", "Israel", "Luxembourg",
    "Iceland", "Czech Republic", "Estonia", "Slovenia", "Portugal","Spain", "Italy", "Czechia", "Korea"
]

developing = [
    "India", "China", "Brazil", "Russia", "Mexico", "South Africa", "Indonesia", "Malaysia", "Turkey",
    "Argentina", "Thailand", "Vietnam", "Chile", "Colombia", "Poland", "Hungary", "Slovakia", "Lithuania",
    "Latvia", "Kazakhstan", "Bulgaria", "Romania", "Peru", "Uruguay", "Panama", "United Arab Emirates",
    "Costa Rica", "Qatar", "Oman", "Saudi Arabia", "Malta", "Bahrain", "Kuwait","Egypt", "Pakistan", "Ethiopia", "Iraq", "Kenya", "Jordan", "Iran",
    "Cameroon", "Greece", "Philippines", "Tanzania", "Kyrgyzstan", "Senegal", "Tunisia",
    "Belarus", "Croatia", "Cyprus", "Lebanon", "Ghana", "Mauritius", "Jamaica", "Uganda",
    "Serbia", "Armenia", "Guatemala", "Ukraine", "Nigeria", "Algeria", "Cuba", "Bolivia",
    "El Salvador", "Bosnia and Herzegovina", "Venezuela", "Moldova", "Georgia", "Paraguay",
    "Botswana", "Morocco", "Sri Lanka", "Trinidad and Tobago", "Brunei", "Uzbekistan",
    "Honduras", "Ecuador", "Bhutan", "Montenegro", "Albania", "Fiji", "Bahamas", "Suriname",
    "North Macedonia", "Nicaragua", "Mongolia", "Azerbaijan"
]

underdeveloped = [
    "Afghanistan", "Yemen", "Sudan", "Chad", "Somalia", "South Sudan", "Democratic Republic of Congo",
    "Haiti", "Burundi", "Liberia", "Sierra Leone", "Mozambique", "Madagascar", "Central African Republic",
    "Eritrea", "Niger", "Mali", "Burkina Faso", "Guinea", "Guinea-Bissau", "Gambia", "Comoros", "Zambia",
    "Zimbabwe", "Nepal", "Bangladesh", "Myanmar", "Papua New Guinea", "Rwanda", "Congo", "Syria", "Malawi", "Benin", "Gabon", "Cambodia", "Lesotho", "Dominican Republic",
    "Lao People's Democratic Republic", "Namibia", "Guyana", "Libya"
]
#Other Institutions (Territories, Former Countries, and Research Sites)
other_institutions = [
    "Puerto Rico", "Palestinian Territory", "Réunion", "French Guiana", "Macau", "Guadeloupe",
    "Kosovo", "Guam", "Greenland", "French Polynesia", "Faroe Islands", "Saint Kitts and Nevis",
    "Andorra", "San Marino", "Micronesia", "Solomon Islands", "Vanuatu", "Samoa", "Saint Lucia",
    "Grenada", "Barbados", "Antigua and Barbuda", "Liechtenstein", "GSK Investigational Site",
    "Yugoslavia", "The Former Yugoslav Republic of", "New Caledonia", "Northern Mariana Islands", "New Caledonia", "Novartis Investigative Site", "Palestinian Territories", "Cayman Islands","Monaco", "Côte D'Ivoire", "Swaziland"
]


# Define category mappings
def categorize_country(country):
    if country in developed:
        return "Developed"
    elif country in developing:
        return "Developing"
    elif country in underdeveloped:
        return "Underdeveloped"
    elif country in other_institutions:
        return "Other Institutions"
    else:
        return "Unknown"

# Apply country categorization
data['Development Category'] = data['Country'].apply(categorize_country)

In [198]:
data['Development Category'].isna().sum()

0

In [199]:
data['Development Category'].isnull().sum()

0

In [200]:
# Function to compute linguistic features
def extract_text_features(text):
    if pd.isnull(text) or text.strip() == '':
        return pd.Series([0, 0, 0, 0, 0])  # Handle missing values

    # Word Count
    word_count = len(text.split())

    # Character Count (excluding spaces)
    char_count = len(text.replace(" ", ""))

    # Sentence Count (Using TextBlob instead of nltk)
    sentence_count = len(TextBlob(text).sentences)

    # Average Word Length
    avg_word_length = char_count / word_count if word_count > 0 else 0

    # Sentiment Score
    sentiment = TextBlob(text).sentiment.polarity

    return pd.Series([word_count, char_count, sentence_count, avg_word_length, sentiment])

# Apply feature extraction to each column
for col in ['Study Title', 'Brief Summary', 'Outcomes']:
    feature_names = [f"{col}_Word_Count", f"{col}_Char_Count", f"{col}_Sentence_Count",
                     f"{col}_Avg_Word_Length", f"{col}_Sentiment_Score"]

    data[feature_names] = data[col].apply(extract_text_features)

# Verify the extracted features
data.head()

,NCT Number,Study Title,Brief Summary,Interventions,Sex,Funder Type,Study Type,Study Design,Start Date,Locations,...,Brief Summary_Word_Count,Brief Summary_Char_Count,Brief Summary_Sentence_Count,Brief Summary_Avg_Word_Length,Brief Summary_Sentiment_Score,Outcomes_Word_Count,Outcomes_Char_Count,Outcomes_Sentence_Count,Outcomes_Avg_Word_Length,Outcomes_Sentiment_Score
0,NCT03162666,Patient Outcomes Using an Expandable Spacer,This study is a post-market clinical follow-up...,DEVICE: ALTERA,ALL,INDUSTRY,OBSERVATIONAL,Observational Model: |Time Perspective: p,2017-02-28,"Rush University Medical Center, Chicago, Illin...",...,28.0,156.0,2.0,5.571429,0.000000,112.0,769.0,1.0,6.866071,0.000000
1,NCT04312048,the Effect of Isosorbide Mononitrate in Reduci...,The aim of the study is to evaluate the effica...,DRUG: Isosorbide mononitrate|DRUG: placebo,FEMALE,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2020-04-15,"Ahmed Samy, Giza, 11231, Egypt",...,25.0,139.0,1.0,5.560000,0.000000,71.0,340.0,2.0,4.788732,-0.200000
2,NCT03144778,Durvalumab With or Without Tremelimumab in Tre...,This phase I trial studies how well durvalumab...,BIOLOGICAL: Durvalumab|BIOLOGICAL: Tremelimumab,ALL,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2017-07-12,"M D Anderson Cancer Center, Houston, Texas, 77...",...,54.0,309.0,2.0,5.722222,0.000000,169.0,934.0,2.0,5.526627,-0.193229
3,NCT01592721,Radiation and Cetuximab Plus Intratumoral EGFR...,The incorporation of novel targeted therapies ...,BIOLOGICAL: EGFR Antisense DNA,ALL,OTHER,INTERVENTIONAL,Allocation: NA|Intervention Model: SINGLE_GROU...,2013-04-01,"University of Pittsburgh, Pittsburgh, Pennsylv...",...,91.0,523.0,3.0,5.747253,0.156944,199.0,1117.0,4.0,5.613065,0.123438
4,NCT04253613,Laser Biostimulation in Periodontal Treatment,"The use of nonsurgical periodontal treatment, ...",NaN,MALE,OTHER,OBSERVATIONAL,Observational Model: |Time Perspective: p,2013-01-15,NaN,...,154.0,989.0,4.0,6.422078,0.129375,95.0,547.0,4.0,5.757895,-0.031250


In [201]:
data = data.drop(columns=['Combined_Text', 'Outcomes', 'Study Title', 'Brief Summary', 'Study Design'], axis=1)
data.shape

(100, 76)

In [202]:
list(data.columns)

['NCT Number',
 'Interventions',
 'Sex',
 'Funder Type',
 'Study Type',
 'Start Date',
 'Locations',
 'Conditions_Category',
 'Predicted Label_Study Title',
 'Probability Class 0_Study Title',
 'Probability Class 1_Study Title',
 'Predicted Label_Brief Summary',
 'Probability Class 0_Brief Summary',
 'Probability Class 1_Brief Summary',
 'Predicted Label_Conditions',
 'Probability Class 0_Conditions',
 'Probability Class 1_Conditions',
 'Predicted Label_Outcomes',
 'Probability Class 0_Outcomes',
 'Probability Class 1_Outcomes',
 'Start Month',
 'Start Quarter',
 'Num_Collaborators',
 'Collaborator_Type',
 'Condition_Category_old',
 'Sponsor Type',
 'CHILD',
 'ADULT',
 'OLDER_ADULT',
 'Masking',
 'Observational Model',
 'Time Perspective',
 'Masking Details',
 'Allocation_',
 'Allocation_NA',
 'Allocation_NON_RANDOMIZED',
 'Allocation_RANDOMIZED',
 'Allocation_Unknown',
 'Intervention Model_',
 'Intervention Model_CROSSOVER',
 'Intervention Model_FACTORIAL',
 'Intervention Model_PARALL

In [203]:
data = data.drop(columns=['Locations'])

In [204]:
# Get categorical columns (object or category type)
categorical_columns = data.select_dtypes(include=['object', 'category']).columns.tolist()

# Get numerical columns (int, float type)
numerical_columns = data.select_dtypes(include=['number']).columns.tolist()

# Remove 'NCT Number' and 'Study Status' from the lists
categorical_columns = [col for col in categorical_columns if col not in ['NCT Number', 'Study Status', 'data_split', 'data_type']]
numerical_columns = [col for col in numerical_columns if col not in ['NCT Number', 'Study Status', 'data_split', 'data_type', 'Start Month', 'Start Quarter']]

# Explicitly add 'Start Month' and 'Start Quarter' to the categorical columns
categorical_columns.extend(['Start Month', 'Start Quarter'])

# Optionally, print the lists of categorical and numerical columns
print("Categorical Columns:", categorical_columns)
print("Numerical Columns:", numerical_columns)

Categorical Columns: ['Interventions', 'Sex', 'Funder Type', 'Study Type', 'Conditions_Category', 'Start Month', 'Start Quarter', 'Collaborator_Type', 'Condition_Category_old', 'Sponsor Type', 'Masking', 'Observational Model', 'Time Perspective', 'Masking Details', 'Country', 'Development Category', 'Start Month', 'Start Quarter']
Numerical Columns: ['Predicted Label_Study Title', 'Probability Class 0_Study Title', 'Probability Class 1_Study Title', 'Predicted Label_Brief Summary', 'Probability Class 0_Brief Summary', 'Probability Class 1_Brief Summary', 'Predicted Label_Conditions', 'Probability Class 0_Conditions', 'Probability Class 1_Conditions', 'Predicted Label_Outcomes', 'Probability Class 0_Outcomes', 'Probability Class 1_Outcomes', 'Num_Collaborators', 'CHILD', 'ADULT', 'OLDER_ADULT', 'Study Title_Word_Count', 'Study Title_Char_Count', 'Study Title_Sentence_Count', 'Study Title_Avg_Word_Length', 'Study Title_Sentiment_Score', 'Brief Summary_Word_Count', 'Brief Summary_Char_Cou

In [205]:
# List of top 5 countries
top_5_countries = ['United States', 'Unknown', 'France', 'India', 'China']

# Create separate columns for each of the top 5 countries
for country in top_5_countries:
    data[country] = data['Country'].apply(lambda x: 1 if x == country else 0)

data = data.drop(columns=['Country'])

data.head()

,NCT Number,Interventions,Sex,Funder Type,Study Type,Start Date,Conditions_Category,Predicted Label_Study Title,Probability Class 0_Study Title,Probability Class 1_Study Title,...,Outcomes_Word_Count,Outcomes_Char_Count,Outcomes_Sentence_Count,Outcomes_Avg_Word_Length,Outcomes_Sentiment_Score,United States,Unknown,France,India,China
0,NCT03162666,DEVICE: ALTERA,ALL,INDUSTRY,OBSERVATIONAL,2017-02-28,Non-Oncology,0,0.978512,0.021488,...,112.0,769.0,1.0,6.866071,0.000000,1,0,0,0,0
1,NCT04312048,DRUG: Isosorbide mononitrate|DRUG: placebo,FEMALE,OTHER,INTERVENTIONAL,2020-04-15,Non-Oncology,1,0.001262,0.998738,...,71.0,340.0,2.0,4.788732,-0.200000,0,0,0,0,0
2,NCT03144778,BIOLOGICAL: Durvalumab|BIOLOGICAL: Tremelimumab,ALL,OTHER,INTERVENTIONAL,2017-07-12,Oncology,0,0.952378,0.047622,...,169.0,934.0,2.0,5.526627,-0.193229,1,0,0,0,0
3,NCT01592721,BIOLOGICAL: EGFR Antisense DNA,ALL,OTHER,INTERVENTIONAL,2013-04-01,Oncology,0,0.613021,0.386979,...,199.0,1117.0,4.0,5.613065,0.123438,1,0,0,0,0
4,NCT04253613,NaN,MALE,OTHER,OBSERVATIONAL,2013-01-15,Non-Oncology,1,0.002673,0.997327,...,95.0,547.0,4.0,5.757895,-0.031250,0,1,0,0,0


In [206]:
categorical_columns = [col for col in categorical_columns if col not in ['NCT Number', 'Study Status', 'data_split', 'data_type', 'Country', 'Study Type', 'categorical_columns']]
numerical_columns.extend(['United States', 'Unknown', 'France', 'India', 'China'])

In [207]:
list(data.columns)

['NCT Number',
 'Interventions',
 'Sex',
 'Funder Type',
 'Study Type',
 'Start Date',
 'Conditions_Category',
 'Predicted Label_Study Title',
 'Probability Class 0_Study Title',
 'Probability Class 1_Study Title',
 'Predicted Label_Brief Summary',
 'Probability Class 0_Brief Summary',
 'Probability Class 1_Brief Summary',
 'Predicted Label_Conditions',
 'Probability Class 0_Conditions',
 'Probability Class 1_Conditions',
 'Predicted Label_Outcomes',
 'Probability Class 0_Outcomes',
 'Probability Class 1_Outcomes',
 'Start Month',
 'Start Quarter',
 'Num_Collaborators',
 'Collaborator_Type',
 'Condition_Category_old',
 'Sponsor Type',
 'CHILD',
 'ADULT',
 'OLDER_ADULT',
 'Masking',
 'Observational Model',
 'Time Perspective',
 'Masking Details',
 'Allocation_',
 'Allocation_NA',
 'Allocation_NON_RANDOMIZED',
 'Allocation_RANDOMIZED',
 'Allocation_Unknown',
 'Intervention Model_',
 'Intervention Model_CROSSOVER',
 'Intervention Model_FACTORIAL',
 'Intervention Model_PARALLEL',
 'Interve

In [208]:
data['Conditions_Category']


0                   Non-Oncology
1                   Non-Oncology
2                       Oncology
3                       Oncology
4                   Non-Oncology
                 ...            
95                  Non-Oncology
96                  Non-Oncology
97                  Non-Oncology
98    Other Rare or Unclassified
99                  Non-Oncology
Name: Conditions_Category, Length: 100, dtype: object

In [209]:
# Load saved OneHotEncoder
encoder = joblib.load(folder_path + 'one_hot_encoder.pkl')

# Load the original feature names from training
with open(folder_path + 'one_hot_encoded_feature_names.json', 'r') as f:
    original_feature_names = json.load(f)

# Identify categorical columns used during training
categorical_columns = list(encoder.feature_names_in_)

# Ensure categorical columns exist in new data
missing_features = [col for col in categorical_columns if col not in data.columns]
if missing_features:
    raise ValueError(f"Missing categorical columns in new data: {missing_features}")

# Transform categorical columns
encoded_columns = encoder.transform(data[categorical_columns])

# Create a DataFrame for the encoded categorical features
encoded_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(categorical_columns))

# Drop original categorical columns and retain numerical ones
numerical_columns = [col for col in data.columns if col not in categorical_columns]  # Keep numerical features
new_data_encoded = data[numerical_columns].reset_index(drop=True)
new_data_encoded = pd.concat([new_data_encoded, encoded_df], axis=1)

# Ensure all expected features are present (fill missing ones with 0)
for feature in original_feature_names:
    if feature not in new_data_encoded.columns:
        new_data_encoded[feature] = 0  # Add missing columns with default value

# Ensure correct column order before passing to model
new_data_encoded = new_data_encoded[original_feature_names]

# **Add 'Study Type' back to the final dataset**
if 'Study Type' in data.columns:
    new_data_encoded['Study Type'] = data['Study Type'].values
else:
    print("Warning: 'Study Type' column not found in original data.")

# **Add 'Study Type' back to the final dataset**
if 'Conditions_Category' in data.columns:
    new_data_encoded['Conditions_Category'] = data['Conditions_Category'].values
else:
    print("Warning: 'Conditions_Category' column not found in original data.")

# Print final feature verification
print(f"Final Transformed Data Columns: {list(new_data_encoded.columns)}")
new_data_encoded.head()

Final Transformed Data Columns: ['NCT Number', 'Study Status', 'Enrollment', 'Num_Collaborators', 'CHILD', 'ADULT', 'OLDER_ADULT', 'Allocation_', 'Allocation_NA', 'Allocation_NON_RANDOMIZED', 'Allocation_RANDOMIZED', 'Allocation_Unknown', 'Intervention Model_', 'Intervention Model_CROSSOVER', 'Intervention Model_FACTORIAL', 'Intervention Model_PARALLEL', 'Intervention Model_SEQUENTIAL', 'Intervention Model_SINGLE_GROUP', 'Intervention Model_Unknown', 'Masking Level_DOUBLE', 'Masking Level_NONE', 'Masking Level_QUADRUPLE', 'Masking Level_SINGLE', 'Masking Level_TRIPLE', 'Primary Purpose_', 'Primary Purpose_BASIC_SCIENCE', 'Primary Purpose_DEVICE_FEASIBILITY', 'Primary Purpose_DIAGNOSTIC', 'Primary Purpose_ECT', 'Primary Purpose_HEALTH_SERVICES_RESEARCH', 'Primary Purpose_OTHER', 'Primary Purpose_PREVENTION', 'Primary Purpose_SCREENING', 'Primary Purpose_SUPPORTIVE_CARE', 'Primary Purpose_TREATMENT', 'Primary Purpose_Unknown', 'Study Title_Word_Count', 'Study Title_Char_Count', 'Study Ti

c:\Users\mogal\anaconda3\envs\ryzenai-1.1-20240613-020240\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.6.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


,NCT Number,Study Status,Enrollment,Num_Collaborators,CHILD,ADULT,OLDER_ADULT,Allocation_,Allocation_NA,Allocation_NON_RANDOMIZED,...,Start Month_9,Start Month_10,Start Month_11,Start Month_12,Start Quarter_1,Start Quarter_2,Start Quarter_3,Start Quarter_4,Study Type,Conditions_Category
0,NCT03162666,0,0,0,0,1,1,False,False,False,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,OBSERVATIONAL,Non-Oncology
1,NCT04312048,0,0,0,1,1,1,False,False,False,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,INTERVENTIONAL,Non-Oncology
2,NCT03144778,0,0,1,0,1,1,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,INTERVENTIONAL,Oncology
3,NCT01592721,0,0,1,0,1,1,False,True,False,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,INTERVENTIONAL,Oncology
4,NCT04253613,0,0,0,0,1,1,False,False,False,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,OBSERVATIONAL,Non-Oncology


In [210]:
categorical_columns

['Sex',
 'Funder Type',
 'Study Type',
 'Conditions_Category',
 'Collaborator_Type',
 'Condition_Category_old',
 'Sponsor Type',
 'Masking',
 'Observational Model',
 'Time Perspective',
 'Masking Details',
 'Development Category',
 'Start Month',
 'Start Quarter']

In [211]:
numerical_columns

['NCT Number',
 'Interventions',
 'Start Date',
 'Predicted Label_Study Title',
 'Probability Class 0_Study Title',
 'Probability Class 1_Study Title',
 'Predicted Label_Brief Summary',
 'Probability Class 0_Brief Summary',
 'Probability Class 1_Brief Summary',
 'Predicted Label_Conditions',
 'Probability Class 0_Conditions',
 'Probability Class 1_Conditions',
 'Predicted Label_Outcomes',
 'Probability Class 0_Outcomes',
 'Probability Class 1_Outcomes',
 'Num_Collaborators',
 'CHILD',
 'ADULT',
 'OLDER_ADULT',
 'Allocation_',
 'Allocation_NA',
 'Allocation_NON_RANDOMIZED',
 'Allocation_RANDOMIZED',
 'Allocation_Unknown',
 'Intervention Model_',
 'Intervention Model_CROSSOVER',
 'Intervention Model_FACTORIAL',
 'Intervention Model_PARALLEL',
 'Intervention Model_SINGLE_GROUP',
 'Intervention Model_Unknown',
 'Masking Level_DOUBLE',
 'Masking Level_NONE',
 'Masking Level_QUADRUPLE',
 'Masking Level_SINGLE',
 'Masking Level_TRIPLE',
 'Primary Purpose_',
 'Primary Purpose_BASIC_SCIENCE',
 

In [212]:
new_data_encoded['Study Type']

0      OBSERVATIONAL
1     INTERVENTIONAL
2     INTERVENTIONAL
3     INTERVENTIONAL
4      OBSERVATIONAL
           ...      
95    INTERVENTIONAL
96    INTERVENTIONAL
97    INTERVENTIONAL
98    INTERVENTIONAL
99    INTERVENTIONAL
Name: Study Type, Length: 100, dtype: object

In [213]:
new_data_encoded['Conditions_Category']

0                   Non-Oncology
1                   Non-Oncology
2                       Oncology
3                       Oncology
4                   Non-Oncology
                 ...            
95                  Non-Oncology
96                  Non-Oncology
97                  Non-Oncology
98    Other Rare or Unclassified
99                  Non-Oncology
Name: Conditions_Category, Length: 100, dtype: object

In [214]:
# import pandas as pd
# import joblib

# # Load the saved OneHotEncoder
# encoder = joblib.load(folder_path + 'one_hot_encoder.pkl')

# # Ensure categorical columns exist in new_data
# categorical_columns = encoder.feature_names_in_  # Fetch the column names used during training

# # Transform the categorical columns in new_data
# encoded_columns = encoder.transform(data[categorical_columns])

# # Create a DataFrame with the one-hot encoded values
# encoded_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(categorical_columns))

# # Drop the original categorical columns and append the encoded columns
# new_data_encoded = data.drop(columns=categorical_columns, errors='ignore')
# new_data_encoded = pd.concat([new_data_encoded, encoded_df], axis=1)

# # Display the first few rows of the transformed data
# print(new_data_encoded.head())


In [215]:
for col in new_data_encoded.columns:
    print(f"{col}: {new_data_encoded[col].iloc[0]}")

NCT Number: NCT03162666
Study Status: 0
Enrollment: 0
Num_Collaborators: 0
CHILD: 0
ADULT: 1
OLDER_ADULT: 1
Allocation_: False
Allocation_NA: False
Allocation_NON_RANDOMIZED: False
Allocation_RANDOMIZED: False
Allocation_Unknown: True
Intervention Model_: False
Intervention Model_CROSSOVER: False
Intervention Model_FACTORIAL: False
Intervention Model_PARALLEL: False
Intervention Model_SEQUENTIAL: 0
Intervention Model_SINGLE_GROUP: False
Intervention Model_Unknown: True
Masking Level_DOUBLE: False
Masking Level_NONE: True
Masking Level_QUADRUPLE: False
Masking Level_SINGLE: False
Masking Level_TRIPLE: False
Primary Purpose_: False
Primary Purpose_BASIC_SCIENCE: False
Primary Purpose_DEVICE_FEASIBILITY: 0
Primary Purpose_DIAGNOSTIC: False
Primary Purpose_ECT: 0
Primary Purpose_HEALTH_SERVICES_RESEARCH: False
Primary Purpose_OTHER: False
Primary Purpose_PREVENTION: False
Primary Purpose_SCREENING: False
Primary Purpose_SUPPORTIVE_CARE: False
Primary Purpose_TREATMENT: False
Primary Purpos

In [216]:
# Define subsets based on Study Type and Conditions_Category
model_interventional_oncology = new_data_encoded[
    (new_data_encoded['Study Type'] == 'INTERVENTIONAL') & (new_data_encoded['Conditions_Category'] == 'Oncology')
]

model_interventional_non_oncology = new_data_encoded[
    (new_data_encoded['Study Type'] == 'INTERVENTIONAL') & (new_data_encoded['Conditions_Category'] == 'Non-Oncology')
]

model_interventional_other = new_data_encoded[
    (new_data_encoded['Study Type'] == 'INTERVENTIONAL') & (new_data_encoded['Conditions_Category'] == 'Other Rare or Unclassified')
]

model_observational_oncology = new_data_encoded[
    (new_data_encoded['Study Type'] == 'OBSERVATIONAL') & (new_data_encoded['Conditions_Category'] == 'Oncology')
]

model_observational_non_oncology = new_data_encoded[
    (new_data_encoded['Study Type'] == 'OBSERVATIONAL') & (new_data_encoded['Conditions_Category'] == 'Non-Oncology')
]

model_observational_other = new_data_encoded[
    (new_data_encoded['Study Type'] == 'OBSERVATIONAL') & (new_data_encoded['Conditions_Category'] == 'Other Rare or Unclassified')
]

# Print counts for each dataset
print(f"Model 1 - Interventional Oncology: {len(model_interventional_oncology)}")
print(f"Model 2 - Interventional Non-Oncology: {len(model_interventional_non_oncology)}")
print(f"Model 3 - Interventional Other: {len(model_interventional_other)}")
print(f"Model 4 - Observational Oncology: {len(model_observational_oncology)}")
print(f"Model 5 - Observational Non-Oncology: {len(model_observational_non_oncology)}")
print(f"Model 6 - Observational Other: {len(model_observational_other)}")

Model 1 - Interventional Oncology: 11
Model 2 - Interventional Non-Oncology: 53
Model 3 - Interventional Other: 19
Model 4 - Observational Oncology: 0
Model 5 - Observational Non-Oncology: 10
Model 6 - Observational Other: 7


In [217]:
# Set 'data_type' for train datasets
model_interventional_non_oncology['data_type'] = 'model_interventional_non_oncology'
model_interventional_oncology['data_type'] = 'model_interventional_oncology'
model_interventional_other['data_type'] = 'model_interventional_other'
model_observational_non_oncology['data_type'] = 'model_observational_non_oncology'
model_observational_oncology['data_type'] = 'model_observational_oncology'
model_observational_other['data_type'] = 'model_observational_other'

C:\Users\mogal\AppData\Local\Temp\ipykernel_42584\2369043078.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_interventional_non_oncology['data_type'] = 'model_interventional_non_oncology'
C:\Users\mogal\AppData\Local\Temp\ipykernel_42584\2369043078.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_interventional_oncology['data_type'] = 'model_interventional_oncology'
C:\Users\mogal\AppData\Local\Temp\ipykernel_42584\2369043078.py:4: SettingWithCopyWarning: 
A value is trying to be set on a

In [218]:
# Combine all the DataFrames into one single dataset
combined_data = pd.concat([
    model_interventional_non_oncology, model_interventional_oncology, model_interventional_other,
    model_observational_non_oncology, model_observational_oncology, model_observational_other,
], ignore_index=True)

# Drop columns that start with 'Unnamed'
combined_data = combined_data.loc[:, ~combined_data.columns.str.startswith('Unnamed')]

combined_data.head()

,NCT Number,Study Status,Enrollment,Num_Collaborators,CHILD,ADULT,OLDER_ADULT,Allocation_,Allocation_NA,Allocation_NON_RANDOMIZED,...,Start Month_9,Start Month_10,Start Month_11,Start Month_12,Start Quarter_1,Start Quarter_2,Start Quarter_3,Start Quarter_4,Study Type,Conditions_Category
0,NCT04312048,0,0,0,1,1,1,False,False,False,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,INTERVENTIONAL,Non-Oncology
1,NCT01728441,0,0,1,0,1,1,False,False,False,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,INTERVENTIONAL,Non-Oncology
2,NCT00835796,0,0,0,0,1,1,False,False,False,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,INTERVENTIONAL,Non-Oncology
3,NCT01284699,0,0,0,0,1,1,False,True,False,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,INTERVENTIONAL,Non-Oncology
4,NCT00316043,0,0,1,1,0,0,False,False,False,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,INTERVENTIONAL,Non-Oncology


In [219]:
import xgboost as xgb
import pandas as pd
import numpy as np
import json
import os
import joblib

# Define directory containing saved models and features
model_dir = folder_path+ '/models'

# List of data types (same as used during training)
data_types = [
    'model_interventional_non_oncology', 'model_interventional_oncology', 'model_interventional_other',
    'model_observational_non_oncology', 'model_observational_oncology', 'model_observational_other'
]


# Initialize list to store results
inference_results = []

# Loop over each model corresponding to a data type
for data_type in data_types:
    # Load the trained model
    model_filename = os.path.join(model_dir, f"{data_type}_xgboost_model.json")
    model = xgb.Booster()
    model.load_model(model_filename)

    # Load feature names used during training
    feature_filename = os.path.join(model_dir, f"{data_type}_features.json")
    with open(feature_filename, 'r') as f:
        feature_names = json.load(f)

    # Filter the new test data to match the data type
    test_subset = combined_data[combined_data['data_type'] == data_type].copy()
    
    # Ensure test data contains the necessary features
    missing_features = [col for col in feature_names if col not in test_subset.columns]
    if missing_features:
        raise ValueError(f"Missing expected features for {data_type}: {missing_features}")

    # Select only the trained model features
    X_test = test_subset[feature_names]

    # Convert to XGBoost DMatrix
    dtest = xgb.DMatrix(X_test)

    # Make predictions
    y_pred_proba = model.predict(dtest)  # Probabilities

    # Ensure binary classification probability format
    if y_pred_proba.ndim == 1:  # Single-class probabilities
        y_pred_proba = np.vstack([1 - y_pred_proba, y_pred_proba]).T  # Convert to two-column format

    # Convert probabilities to class predictions
    y_pred_class = np.round(y_pred_proba[:, 1])  # Class 1 prediction threshold at 0.5

    # Create DataFrame for predictions
    predictions_df = pd.DataFrame({
        "NCT Number": test_subset["NCT Number"],
        "Predicted Class": y_pred_class,
        "Probability Class 0": y_pred_proba[:, 0],
        "Probability Class 1": y_pred_proba[:, 1],
        "data_type": data_type
    })

    # Append to results
    inference_results.append(predictions_df)

# Combine all results into a single DataFrame
final_predictions = pd.concat(inference_results, ignore_index=True)

final_predictions.head()


c:\Users\mogal\anaconda3\envs\ryzenai-1.1-20240613-020240\lib\site-packages\xgboost\core.py:158: UserWarning: [18:00:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:52: Empty dataset at worker: 0
  warnings.warn(smsg, UserWarning)


,NCT Number,Predicted Class,Probability Class 0,Probability Class 1,data_type
0,NCT04312048,1.0,0.206611,0.793389,model_interventional_non_oncology
1,NCT01728441,0.0,0.575113,0.424887,model_interventional_non_oncology
2,NCT00835796,1.0,0.153820,0.846180,model_interventional_non_oncology
3,NCT01284699,0.0,0.642036,0.357964,model_interventional_non_oncology
4,NCT00316043,1.0,0.434994,0.565006,model_interventional_non_oncology


In [220]:
filtered_data = data.filter(regex=r'^(Predicted Label|NCT Number)', axis=1)

In [221]:
merged_data= pd.merge(filtered_data, final_predictions, on='NCT Number', how='inner')

In [222]:
merged_data

,NCT Number,Predicted Label_Study Title,Predicted Label_Brief Summary,Predicted Label_Conditions,Predicted Label_Outcomes,Predicted Class,Probability Class 0,Probability Class 1,data_type
0,NCT03162666,0,1,0,0,0.0,0.712114,0.287886,model_observational_non_oncology
1,NCT04312048,1,1,1,1,1.0,0.206611,0.793389,model_interventional_non_oncology
2,NCT03144778,0,0,0,0,1.0,0.400128,0.599872,model_interventional_oncology
3,NCT01592721,0,0,0,0,0.0,0.508016,0.491984,model_interventional_oncology
4,NCT04253613,1,1,1,1,0.0,0.582175,0.417825,model_observational_non_oncology
...,...,...,...,...,...,...,...,...,...
95,NCT04320212,1,1,0,1,1.0,0.156323,0.843677,model_interventional_non_oncology
96,NCT04123054,1,1,1,1,0.0,0.539119,0.460881,model_interventional_non_oncology
97,NCT05887947,1,1,1,1,0.0,0.700656,0.299344,model_interventional_non_oncology
98,NCT02235961,1,1,1,1,1.0,0.325922,0.674078,model_interventional_other


In [223]:
columns_to_keep = [
    'NCT Number',
    'Predicted Label_Study Title',
    'Predicted Label_Brief Summary',
    'Predicted Label_Conditions',
    'Predicted Label_Outcomes',
    'Predicted Class'
]

# Select only the desired columns
final_data = merged_data[columns_to_keep]

In [224]:
final_data

,NCT Number,Predicted Label_Study Title,Predicted Label_Brief Summary,Predicted Label_Conditions,Predicted Label_Outcomes,Predicted Class
0,NCT03162666,0,1,0,0,0.0
1,NCT04312048,1,1,1,1,1.0
2,NCT03144778,0,0,0,0,1.0
3,NCT01592721,0,0,0,0,0.0
4,NCT04253613,1,1,1,1,0.0
...,...,...,...,...,...,...
95,NCT04320212,1,1,0,1,1.0
96,NCT04123054,1,1,1,1,0.0
97,NCT05887947,1,1,1,1,0.0
98,NCT02235961,1,1,1,1,1.0


In [225]:
# Initialize a list to store the final majority voting predictions
final_predictions = []

# Iterate through each row in the test data
for index, row in final_data.iterrows():
  # Extract predicted labels
  pred_conditions = row['Predicted Label_Conditions']
  pred_study_title = row['Predicted Label_Study Title']
  pred_outcomes = row['Predicted Label_Outcomes']
  pred_brief_summary = row['Predicted Label_Brief Summary']
  pred_others = row['Predicted Class']

# Apply the given logic:
  if pred_others == 0:
      final_predictions.append(0)
  else:
      # Check if 4/5 of the remaining predictions are 0
      remaining_predictions = [pred_study_title, pred_outcomes, pred_brief_summary, pred_conditions]
      zero_count = remaining_predictions.count(0)
      
      if zero_count >= 3:  # 4 or more predictions are 0
          final_predictions.append(0)
      else:
          final_predictions.append(1)

# Append the final predictions as a new column to "final_data"
final_data["Final Prediction (Heuristic)"] = final_predictions


C:\Users\mogal\AppData\Local\Temp\ipykernel_42584\2414733733.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data["Final Prediction (Heuristic)"] = final_predictions


In [226]:
final_data

,NCT Number,Predicted Label_Study Title,Predicted Label_Brief Summary,Predicted Label_Conditions,Predicted Label_Outcomes,Predicted Class,Final Prediction (Heuristic)
0,NCT03162666,0,1,0,0,0.0,0
1,NCT04312048,1,1,1,1,1.0,1
2,NCT03144778,0,0,0,0,1.0,0
3,NCT01592721,0,0,0,0,0.0,0
4,NCT04253613,1,1,1,1,0.0,0
...,...,...,...,...,...,...,...
95,NCT04320212,1,1,0,1,1.0,1
96,NCT04123054,1,1,1,1,0.0,0
97,NCT05887947,1,1,1,1,0.0,0
98,NCT02235961,1,1,1,1,1.0,1


In [228]:
Results = final_data[['NCT Number','Final Prediction (Heuristic)']]

In [111]:
Results.to_csv(folder_path+'\Results.csv')